### here i am trying to retain header and the data shape together


Imports

In [175]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace,lit, desc, lower
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql import SparkSession

Starting session

In [176]:
spark = SparkSession.builder \
    .appName("Clean dataset") \
    .getOrCreate()
#spark = SparkSession.builder \
    #.appName("Clean dataset 3").config("spark.executor.instances", "3").config("spark.executor.cores", "2").config("spark.executor.memory", "4g").getOrCreate()

paths to csv and textfile

In [177]:
text_path = "hdfs:///eaplayer_data/male_players_text_revised"

In [178]:
text_rdd = spark.sparkContext.textFile(text_path)

In [179]:
header = text_rdd.filter(lambda row: "player_id" in row).first()
data_rdd = text_rdd.filter(lambda row: row != header)

ValueError: RDD is empty

In [ ]:
data_rdd.take(5)

AttributeError: 'NoneType' object has no attribute 'sc'

Debugging none value

In [ ]:
def has_none_values(fields):
    return any(field.strip() == "" or field.strip().lower() == "none" for field in fields)
none_rows = data_rdd.filter(has_none_values)
none_count = none_rows.count()
print(f"Number of rows with None values: {none_count}")

Number of rows with None values: 10003590


In [ ]:
split_header = header.split(",")
split_header

['player_id',
 'player_url',
 'fifa_version',
 'fifa_update',
 'fifa_update_date',
 'short_name',
 'long_name',
 'player_positions',
 'overall',
 'potential',
 'value_eur',
 'wage_eur',
 'age',
 'dob',
 'height_cm',
 'weight_kg',
 'league_id',
 'league_name',
 'league_level',
 'club_team_id',
 'club_name',
 'club_position',
 'club_jersey_number',
 'club_loaned_from',
 'club_joined_date',
 'club_contract_valid_until_year',
 'nationality_id',
 'nationality_name',
 'nation_team_id',
 'nation_position',
 'nation_jersey_number',
 'preferred_foot',
 'weak_foot',
 'skill_moves',
 'international_reputation',
 'work_rate',
 'body_type',
 'real_face',
 'release_clause_eur',
 'player_tags',
 'player_traits',
 'pace',
 'shooting',
 'passing',
 'dribbling',
 'defending',
 'physic',
 'attacking_crossing',
 'attacking_finishing',
 'attacking_heading_accuracy',
 'attacking_short_passing',
 'attacking_volleys',
 'skill_dribbling',
 'skill_curve',
 'skill_fk_accuracy',
 'skill_long_passing',
 'skill_bal

Spliting all rows into little strings

In [ ]:
text_rdd_split = data_rdd.map(lambda x: x.split(", "))

Checking the available players for the every version

In [ ]:
number_players =text_rdd_split.map(lambda x:(x[2],1))
number_players.reduceByKey(lambda x,y: x+y).collect()

[('21', 1197628),
 ('20', 1128336),
 ('23', 166674),
 ('18', 1512360),
 ('16', 986175),
 ('15', 962549),
 ('19', 1229986),
 ('22', 1218451),
 ('17', 1601431)]

Filtering the latest version

In [ ]:
version_23 = text_rdd_split.filter(lambda x: x[2] == '23')
version_23.count()

166674

checking duplicates

In [ ]:
rdd = version_23.distinct()

checking number of columns

In [ ]:
rdd = rdd.filter(lambda cols: len(cols) == 110 )

removing less important columns 

In [ ]:
columns_to_drop = ['player_url', 
'player_face_url', 
'player_traits', 
'real_face', 
'release_clause_eur', 
'body_type', 
'nation_jersey_number', 
'club_loaned_from', 
'goalkeeping_speed', 
'fifa_update', 
'fifa_update_date', 
'long_name', 
'wage_eur', 
'value_eur', 
'dob', 
'player_tags', 
'club_name',
'player_positions',
'club_jersey_number',
'club_joined_date',
'club_contract_valid_until_year',
'league_id',
 'league_name',
 'league_level',
 'club_team_id',
 'nationality_id',
 'nationality_name',
 'nation_team_id',
 'nation_position',
 'preferred_foot',
 'international_reputation',
 'work_rate',]

In [ ]:
drop_indices = [i for i, col in enumerate(split_header) if col in columns_to_drop]
print("Indices of columns to drop:", drop_indices)

Indices of columns to drop: [1, 3, 4, 6, 7, 10, 11, 13, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 34, 35, 36, 37, 38, 39, 40, 81, 109]


In [ ]:
keep_indices = [i for i in range(len(split_header)) if i not in drop_indices]
print("Indices of columns to keep:", keep_indices)

Indices of columns to keep: [0, 2, 5, 8, 9, 12, 14, 15, 21, 32, 33, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108]


In [ ]:
text_rdd_selected = version_23.map(lambda row: [row[i] for i in keep_indices])
selected_header_columns = [split_header[i] for i in keep_indices]

In [ ]:
len(selected_header_columns)

78

Removing copies, where id is same but other data is different. 

In [ ]:
keyed_rdd = text_rdd_selected.map(lambda row: (row[0], row))
deduplicated_rdd = keyed_rdd.reduceByKey(lambda a, b: a)
final_rdd = deduplicated_rdd.map(lambda kv: kv[1])

In [ ]:
original_unique_count = text_rdd_selected.map(lambda row: row[0]).distinct().count()
deduplicated_unique_count = final_rdd.map(lambda row: row[0]).distinct().count()
print(f"Unique player_ids in original RDD: {original_unique_count}")
print(f"Unique player_ids in deduplicated RDD: {deduplicated_unique_count}")

Unique player_ids in original RDD: 20621
Unique player_ids in deduplicated RDD: 20621


Search a single ID to see if we have duplicates. 

In [ ]:
search_player_id = '189881'
matching_players = final_rdd.filter(lambda row: row[0] == search_player_id)
results = matching_players.collect()
for player in results:
    print(player)

['189881', '23', 'C. Smalling', '81', '81', '32', '194', '81', 'CB', '3', '2', '62', '47', '58', '60', '83', '80', '52', '54', '84', '65', '49', '59', '49', '22', '64', '61', '52', '71', '54', '79', '43', '42', '80', '69', '85', '35', '81', '79', '46', '56', '52', '80', '84', '85', '80', '10', '15', '16', '15', '15', '60+3', '60+3', '60+3', '58', '58', '58', '58', '58', '59+3', '59+3', '59+3', '60+3', '63+3', '63+3', '63+3', '60+3', '71+3', '74+3', '74+3', '74+3', '71+3', '73+3', '81', '81', '81', '73+3', '21+3']


Searching a single name to see if we have duplicates

In [ ]:
search_short_name = 'Azpilicueta' 
matching_players = final_rdd.filter(lambda row: row[2] == search_short_name)
results = matching_players.collect()
for player in results:
    print(player)

['184432', '23', 'Azpilicueta', '82', '82', '32', '178', '77', 'RWB', '3', '2', '64', '55', '76', '72', '84', '75', '80', '46', '78', '79', '51', '68', '68', '58', '79', '77', '68', '61', '67', '86', '73', '68', '83', '75', '70', '60', '87', '83', '62', '72', '66', '78', '85', '85', '84', '13', '9', '10', '6', '5', '66+3', '66+3', '66+3', '69', '69', '69', '69', '69', '71+3', '71+3', '71+3', '72+3', '75+3', '75+3', '75+3', '72+3', '79+3', '81+1', '81+1', '81+1', '79+3', '80+2', '81+1', '81+1', '81+1', '80+2', '17+3']


Adding the values inside a few columns, '80+3' to '83'

In [ ]:
def evaluate_expression(row):
    processed_row = []
    for element in row:
        if '+' in element:
            value, increment = element.split('+')
            result = int(value) + int(increment)
            processed_row.append(str(result))
        else:
            processed_row.append(element)
    return processed_row
processed_rdd = final_rdd.map(evaluate_expression)

In [ ]:
for row in processed_rdd.take(5):
    print(row)

['239890', '23', 'O. Kabak', '75', '81', '22', '186', '86', 'RCB', '3', '2', '69', '36', '55', '66', '74', '80', '26', '35', '76', '69', '28', '64', '32', '31', '67', '71', '63', '74', '64', '66', '64', '48', '85', '67', '85', '25', '84', '68', '30', '62', '51', '67', '73', '77', '73', '10', '13', '10', '6', '5', '57', '57', '57', '55', '56', '56', '56', '55', '61', '61', '61', '60', '65', '65', '65', '60', '67', '73', '73', '73', '67', '69', '77', '77', '77', '69', '17']
['240223', '23', 'M. Ristić', '74', '75', '26', '180', '79', 'SUB', '3', '3', '75', '63', '66', '73', '69', '68', '74', '58', '64', '70', '48', '74', '63', '44', '62', '75', '76', '74', '69', '71', '60', '78', '64', '77', '66', '67', '62', '69', '67', '59', '46', '67', '68', '70', '71', '7', '8', '8', '14', '5', '70', '70', '70', '70', '69', '69', '69', '70', '71', '71', '71', '73', '70', '70', '70', '73', '74', '71', '71', '71', '74', '74', '70', '70', '70', '74', '17']
['207753', '23', 'Oh Ban Suk', '68', '68', '34'

#### lowercase the club position

In [ ]:
updated_rdd = processed_rdd.map(lambda row: row[:8] + [row[8].lower()] + row[9:])

In [ ]:
for row in updated_rdd.take(5):
    print(row)

['239890', '23', 'O. Kabak', '75', '81', '22', '186', '86', 'rcb', '3', '2', '69', '36', '55', '66', '74', '80', '26', '35', '76', '69', '28', '64', '32', '31', '67', '71', '63', '74', '64', '66', '64', '48', '85', '67', '85', '25', '84', '68', '30', '62', '51', '67', '73', '77', '73', '10', '13', '10', '6', '5', '57', '57', '57', '55', '56', '56', '56', '55', '61', '61', '61', '60', '65', '65', '65', '60', '67', '73', '73', '73', '67', '69', '77', '77', '77', '69', '17']
['240223', '23', 'M. Ristić', '74', '75', '26', '180', '79', 'sub', '3', '3', '75', '63', '66', '73', '69', '68', '74', '58', '64', '70', '48', '74', '63', '44', '62', '75', '76', '74', '69', '71', '60', '78', '64', '77', '66', '67', '62', '69', '67', '59', '46', '67', '68', '70', '71', '7', '8', '8', '14', '5', '70', '70', '70', '70', '69', '69', '69', '70', '71', '71', '71', '73', '70', '70', '70', '73', '74', '71', '71', '71', '74', '74', '70', '70', '70', '74', '17']
['207753', '23', 'Oh Ban Suk', '68', '68', '34'

In [ ]:
updated_rdd.count()

20621

In [ ]:
def safe_int_conversion(value):
    try:
        return int(value)
    except (ValueError, TypeError):
        return None 

def convert_row(row):
    return (
        safe_int_conversion(row[0]),  # player_id
        safe_int_conversion(row[1]),  # fifa_version
        row[2],                       # short_name
        safe_int_conversion(row[3]),  # overall
        safe_int_conversion(row[4]),  # potential
        safe_int_conversion(row[5]),  # age
        safe_int_conversion(row[6]),  # height_cm
        safe_int_conversion(row[7]),  # weight_kg
        row[8],                       # club_position
        safe_int_conversion(row[9]),  # weak_foot
        safe_int_conversion(row[10]), # skill_moves
        safe_int_conversion(row[11]), # pace
        safe_int_conversion(row[12]), # shooting
        safe_int_conversion(row[13]), # passing
        safe_int_conversion(row[14]), # dribbling
        safe_int_conversion(row[15]), # defending
        safe_int_conversion(row[16]), # physic
        safe_int_conversion(row[17]), # attacking_crossing
        safe_int_conversion(row[18]), # attacking_finishing
        safe_int_conversion(row[19]), # attacking_heading_accuracy
        safe_int_conversion(row[20]), # attacking_short_passing
        safe_int_conversion(row[21]), # attacking_volleys
        safe_int_conversion(row[22]), # skill_dribbling
        safe_int_conversion(row[23]), # skill_curve
        safe_int_conversion(row[24]), # skill_fk_accuracy
        safe_int_conversion(row[25]), # skill_long_passing
        safe_int_conversion(row[26]), # skill_ball_control
        safe_int_conversion(row[27]), # movement_acceleration
        safe_int_conversion(row[28]), # movement_sprint_speed
        safe_int_conversion(row[29]), # movement_agility
        safe_int_conversion(row[30]), # movement_reactions
        safe_int_conversion(row[31]), # movement_balance
        safe_int_conversion(row[32]), # power_shot_power
        safe_int_conversion(row[33]), # power_jumping
        safe_int_conversion(row[34]), # power_stamina
        safe_int_conversion(row[35]), # power_strength
        safe_int_conversion(row[36]), # power_long_shots
        safe_int_conversion(row[37]), # mentality_aggression
        safe_int_conversion(row[38]), # mentality_interceptions
        safe_int_conversion(row[39]), # mentality_positioning
        safe_int_conversion(row[40]), # mentality_vision
        safe_int_conversion(row[41]), # mentality_penalties
        safe_int_conversion(row[42]), # mentality_composure
        safe_int_conversion(row[43]), # defending_marking_awareness
        safe_int_conversion(row[44]), # defending_standing_tackle
        safe_int_conversion(row[45]), # defending_sliding_tackle
        safe_int_conversion(row[46]), # goalkeeping_diving
        safe_int_conversion(row[47]), # goalkeeping_handling
        safe_int_conversion(row[48]), # goalkeeping_kicking
        safe_int_conversion(row[49]), # goalkeeping_positioning
        safe_int_conversion(row[50]), # goalkeeping_reflexes
        safe_int_conversion(row[51]), # ls
        safe_int_conversion(row[52]), # st
        safe_int_conversion(row[53]), # rs
        safe_int_conversion(row[54]), # lw
        safe_int_conversion(row[55]), # lf
        safe_int_conversion(row[56]), # cf
        safe_int_conversion(row[57]), # rf
        safe_int_conversion(row[58]), # rw
        safe_int_conversion(row[59]), # lam
        safe_int_conversion(row[60]), # cam
        safe_int_conversion(row[61]), # ram
        safe_int_conversion(row[62]), # lm
        safe_int_conversion(row[63]), # lcm
        safe_int_conversion(row[64]), # cm
        safe_int_conversion(row[65]), # rcm
        safe_int_conversion(row[66]), # rm
        safe_int_conversion(row[67]), # lwb
        safe_int_conversion(row[68]), # ldm
        safe_int_conversion(row[69]), # cdm
        safe_int_conversion(row[70]), # rdm
        safe_int_conversion(row[71]), # rwb
        safe_int_conversion(row[72]), # lb
        safe_int_conversion(row[73]), # lcb
        safe_int_conversion(row[74]), # cb
        safe_int_conversion(row[75]), # rcb
        safe_int_conversion(row[76]), # rb
        safe_int_conversion(row[77])  # gk
    )

cl_rdd = updated_rdd.map(convert_row)
# cl_rdd = c_rdd.filter(lambda row: None not in row)

Defining Schema

In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

schema = StructType([
    StructField("player_id", IntegerType(), True),
    StructField("fifa_version", IntegerType(), True),
    StructField("short_name", StringType(), True),
    StructField("overall", IntegerType(), True),
    StructField("potential", IntegerType(), True),
    StructField("age", IntegerType(), True),
    StructField("height_cm", IntegerType(), True),
    StructField("weight_kg", IntegerType(), True),
    StructField("club_position", StringType(), True),
    StructField("weak_foot", IntegerType(), True),
    StructField("skill_moves", IntegerType(), True),
    StructField("pace", IntegerType(), True),
    StructField("shooting", IntegerType(), True),
    StructField("passing", IntegerType(), True),
    StructField("dribbling", IntegerType(), True),
    StructField("defending", IntegerType(), True),
    StructField("physic", IntegerType(), True),
    StructField("attacking_crossing", IntegerType(), True),
    StructField("attacking_finishing", IntegerType(), True),
    StructField("attacking_heading_accuracy", IntegerType(), True),
    StructField("attacking_short_passing", IntegerType(), True),
    StructField("attacking_volleys", IntegerType(), True),
    StructField("skill_dribbling", IntegerType(), True),
    StructField("skill_curve", IntegerType(), True),
    StructField("skill_fk_accuracy", IntegerType(), True),
    StructField("skill_long_passing", IntegerType(), True),
    StructField("skill_ball_control", IntegerType(), True),
    StructField("movement_acceleration", IntegerType(), True),
    StructField("movement_sprint_speed", IntegerType(), True),
    StructField("movement_agility", IntegerType(), True),
    StructField("movement_reactions", IntegerType(), True),
    StructField("movement_balance", IntegerType(), True),
    StructField("power_shot_power", IntegerType(), True),
    StructField("power_jumping", IntegerType(), True),
    StructField("power_stamina", IntegerType(), True),
    StructField("power_strength", IntegerType(), True),
    StructField("power_long_shots", IntegerType(), True),
    StructField("mentality_aggression", IntegerType(), True),
    StructField("mentality_interceptions", IntegerType(), True),
    StructField("mentality_positioning", IntegerType(), True),
    StructField("mentality_vision", IntegerType(), True),
    StructField("mentality_penalties", IntegerType(), True),
    StructField("mentality_composure", IntegerType(), True),
    StructField("defending_marking_awareness", IntegerType(), True),
    StructField("defending_standing_tackle", IntegerType(), True),
    StructField("defending_sliding_tackle", IntegerType(), True),
    StructField("goalkeeping_diving", IntegerType(), True),
    StructField("goalkeeping_handling", IntegerType(), True),
    StructField("goalkeeping_kicking", IntegerType(), True),
    StructField("goalkeeping_positioning", IntegerType(), True),
    StructField("goalkeeping_reflexes", IntegerType(), True),
    StructField("ls", IntegerType(), True),
    StructField("st", IntegerType(), True),
    StructField("rs", IntegerType(), True),
    StructField("lw", IntegerType(), True),
    StructField("lf", IntegerType(), True),
    StructField("cf", IntegerType(), True),
    StructField("rf", IntegerType(), True),
    StructField("rw", IntegerType(), True),
    StructField("lam", IntegerType(), True),
    StructField("cam", IntegerType(), True),
    StructField("ram", IntegerType(), True),
    StructField("lm", IntegerType(), True),
    StructField("lcm", IntegerType(), True),
    StructField("cm", IntegerType(), True),
    StructField("rcm", IntegerType(), True),
    StructField("rm", IntegerType(), True),
    StructField("lwb", IntegerType(), True),
    StructField("ldm", IntegerType(), True),
    StructField("cdm", IntegerType(), True),
    StructField("rdm", IntegerType(), True),
    StructField("rwb", IntegerType(), True),
    StructField("lb", IntegerType(), True),
    StructField("lcb", IntegerType(), True),
    StructField("cb", IntegerType(), True),
    StructField("rcb", IntegerType(), True),
    StructField("rb", IntegerType(), True),
    StructField("gk", IntegerType(), True)
])


In [ ]:
# defenderCentralBack = dfVersion_23.filter((col("player_positions") == "cb")& (col("potential").cast(IntegerType()) > 85)  )
# defenderCentralBack.show()

In [ ]:
# DefenderRightCentralBack  = dfVersion_23.filter((col("club_position") == "RCB")& (col("potential").cast(IntegerType()) > 90) & (col("age").cast(IntegerType()) < 30))
# DefenderRightCentralBack.show()

Creating a dataframe from cleaned data

In [ ]:
df = spark.createDataFrame(cl_rdd,schema)

In [ ]:
""" df.repartition(10) """

' df.repartition(10) '

In [ ]:
df.show(5)

+---------+------------+-------------+-------+---------+---+---------+---------+-------------+---------+-----------+----+--------+-------+---------+---------+------+------------------+-------------------+--------------------------+-----------------------+-----------------+---------------+-----------+-----------------+------------------+------------------+---------------------+---------------------+----------------+------------------+----------------+----------------+-------------+-------------+--------------+----------------+--------------------+-----------------------+---------------------+----------------+-------------------+-------------------+---------------------------+-------------------------+------------------------+------------------+--------------------+-------------------+-----------------------+--------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|player_id|fifa_version|   short_name|overall|potent

In [ ]:
#goolKeeper = cleaned_rdd.filter((col("club_position") == "CF") & (col("potential") > 90) & (col("age") < 30) &(col('goalkeeping_reflexes') > 89) & (col("height_cm") > 195))
#goolKeeper.show()

saving data

In [ ]:
""" df.repartition(1).write.parquet('/eaplayer_data/male_players_version_23.parquet', mode='overwrite') """

" df.repartition(1).write.parquet('/eaplayer_data/male_players_version_23.parquet', mode='overwrite') "

In [ ]:
""" df.repartition(1).write.parquet('/eaplayer_data/male_players1.parquet', mode='overwrite') """

In [ ]:
spark.stop()

# following code might be usefull for debugging

### Checking if players have copies (just saw a 29 year old messi)

In [ ]:
""" threshold = 90
total_players = df.count()
over_threshold_count = df.filter(col("short_name") == "L. Messi")
over_threshold_count.take(5) """

' threshold = 90\ntotal_players = df.count()\nover_threshold_count = df.filter(col("short_name") == "L. Messi")\nover_threshold_count.take(5) '

### Checking how many players are over a certian criteria 

In [ ]:
""" threshold = 90
total_players = df.count()
over_threshold_count = df.filter(col("potential").cast(IntegerType()) > threshold)
over_threshold_count.show() """

' threshold = 90\ntotal_players = df.count()\nover_threshold_count = df.filter(col("potential").cast(IntegerType()) > threshold)\nover_threshold_count.show() '